In [4]:
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import os

In [5]:
kBT   = 10
kBT_  = 20
print(kBT,kBT_)

n = 10
aa = np.array([-0.558223634633, 1.4417258418],dtype=np.float64)
bb = np.array([0.623499404931, 0.0280377585287],dtype=np.float64)

sigma = 0.05
def Grad(px):
    aa = [-1,-1,-6.5,0.7]
    bb = [0,0,11,0.6]
    cc = [-10,-10,-6.5,0.7]
    AA = [-200,-100,-170,15]

    XX = [1,0,-0.5,-1]
    YY = [0,0.5,1.5,1]
    
    gg = np.zeros(shape=(n,),dtype=np.float64)
    for j in range(4):
        ee = AA[j]*np.exp(aa[j]*(px[0]-XX[j])**2+bb[j]*(px[0]-XX[j])*(px[1]-YY[j])+cc[j]*(px[1]-YY[j])**2)
        gg[0] = gg[0] + (2*aa[j]*(px[0]-XX[j])+bb[j]*(px[1]-YY[j]))*ee
        gg[1] = gg[1] + (bb[j]*(px[0]-XX[j])+2*cc[j]*(px[1]-YY[j]))*ee
    gg[0] += 9*2*5*np.pi*np.cos(2*5*np.pi*px[0])*np.sin(2*5*np.pi*px[1])
    gg[1] += 9*2*5*np.pi*np.sin(2*5*np.pi*px[0])*np.cos(2*5*np.pi*px[1])
    for i in range(2,n):
        gg[i] = px[i]/sigma**2
    return gg

def V(px):
    aa = [-1,-1,-6.5,0.7]
    bb = [0,0,11,0.6]
    cc = [-10,-10,-6.5,0.7]
    AA = [-200,-100,-170,15]
    XX = [1,0,-0.5,-1]
    YY = [0,0.5,1.5,1]
    ee = 0
    for j in range(4):
        ee =  ee + AA[j]*np.exp(aa[j]*(px[:,0]-XX[j])**2+bb[j]*(px[:,0]-XX[j])*(px[:,1]-YY[j])+cc[j]*(px[:,1]-YY[j])**2)
    ee += 9*np.sin(2*5*np.pi*px[:,0])*np.sin(2*5*np.pi*px[:,1])
    for i in range(2,n):
        ee += px[:,i]**2/2/sigma**2
    return ee

def construct_NN(x,nlayer,nnode):
    q = tf.contrib.layers.fully_connected(x,nnode,tf.nn.tanh,scope ="layer1")
    for i in range(1,nlayer):
        q = tf.contrib.layers.fully_connected(q,nnode,tf.nn.tanh,scope ="layer"+str(i+1))
    q = tf.contrib.layers.fully_connected(q,1,tf.nn.sigmoid,scope="output")
    dA2 = (x[:,0]-aa[0])**2+(x[:,1]-aa[1])**2
    dB2 = (x[:,0]-bb[0])**2+(x[:,1]-bb[1])**2
    IA = 0.5-0.5*tf.nn.tanh( 1000*(dA2-0.1**2)  )
    IB = 0.5-0.5*tf.nn.tanh( 1000*(dB2-0.1**2)  )
    return (1-IA)*(1-IB)*tf.reshape(q,[-1])+(1-IA)*IB

tf.reset_default_graph()

# input
x = tf.placeholder(dtype=tf.float64, shape=[None,n])
coef = tf.placeholder(dtype=tf.float64, shape=[None,])

# q and gradients
q = construct_NN(x,2,20)
g = tf.gradients(q, x)[0]
fn = tf.reduce_mean( tf.reduce_sum(g**2,axis=1)*coef )

# train_step and initialization
init = tf.global_variables_initializer()
saver = tf.train.Saver()
sess = tf.Session()
sess.run(init)

NNfile = "saved_models/Mueller_"+str(kBT)+"_"+str(kBT_)+"_220.ckpt"
saver.restore(sess,NNfile)

10 20
INFO:tensorflow:Restoring parameters from saved_models/Mueller_10_20_220.ckpt


In [6]:
dt = 1e-5
kapa = 30000

def get_q(px):
    return sess.run(q,{x:[px]})[0]

px = np.zeros(dtype=np.float64,shape=(n,))
px[:2] = (aa+bb)/2
for i in range(20000):
    px -= dt*kapa*(sess.run(q,{x:[px]})[0]-0.5)*sess.run(g,{x:[px]})[0]
    if i%1000==0:
        print(i,get_q(px))
    if np.abs(get_q(px)-0.5)<1e-5:
        break
print(px, get_q(px))

# transition state sampling
dt      = 1e-6
nsample = int(2e5)
coor    = []
for i in range(nsample):
    px = px - dt*Grad(px)+ np.sqrt(2*kBT*dt)*np.random.normal(size=(n,)) - dt*kapa*(sess.run(q,{x:[px]})[0]-0.5)*sess.run(g,{x:[px]})[0]
    qval = get_q(px)
    if i>nsample/10 and i%1000==0:
        coor.append(px)
        print(i,px[:2],qval,LA.norm(Grad(px)))
        
print(np.shape(coor))
coor = np.array(coor)

# sample 100 points
cs = coor[np.linspace(0,coor.shape[0]-1,100,dtype=np.int)]
np.savetxt('FEM/coor_220.txt',cs)

0 0.9484929181863248
[-0.03007635  1.06453929 -0.03330791 -0.05879552  0.10143579  0.02631454
 -0.15596874 -0.41587726  0.58596508  0.04961897] 0.5000087575127745
21000 [-0.65729634  0.72549453] 0.49172448866562335 317.49352441137364
22000 [-0.63358545  0.75158412] 0.4788484809789995 162.58755781778956
23000 [-0.58198131  0.78161689] 0.513421501811474 268.181921895777
24000 [-0.69074054  0.698205  ] 0.4784933316112004 222.0530928700092
25000 [-0.77552093  0.56251087] 0.5092256712988379 299.7801361964811
26000 [-0.86311246  0.35393228] 0.5143889224757272 223.87602026139862
27000 [-0.86273693  0.42541025] 0.4728741190176954 356.5913279605196
28000 [-0.81007918  0.49312166] 0.5085837815127123 156.08116058142755
29000 [-0.8489154   0.44659272] 0.4914427261546978 214.24918839187637
30000 [-0.85544327  0.41848898] 0.49123685895444136 361.3230786412361
31000 [-0.84555202  0.43150749] 0.4953542743665047 286.7608138476171
32000 [-0.82979375  0.46540697] 0.5074025068926447 160.62914009871366
330

136000 [-0.67398957  0.72097666] 0.4832271327376957 379.4704292172279
137000 [-0.7777326   0.56162215] 0.5145682522958749 333.919511788472
138000 [-0.81220965  0.49261111] 0.5093065464135386 236.94431556553243
139000 [-0.81627384  0.48726581] 0.4921105213464258 255.28953677727878
140000 [-0.79926351  0.52422477] 0.5024926865706968 309.4396563124217
141000 [-0.77785859  0.56766214] 0.49338688552874965 293.72707865799504
142000 [-0.69461111  0.68122713] 0.4926156835921248 242.33180301707787
143000 [-0.66030979  0.69197878] 0.5471734420432425 419.9633112150511
144000 [-0.55238114  0.80634886] 0.5072123608908686 431.5405889197699
145000 [-0.57873917  0.78621714] 0.5088770357068809 318.615094511674
146000 [-0.58786827  0.7822477 ] 0.49811535862529593 249.4373649744671
147000 [-0.4831412  0.8397698] 0.5303623726518948 246.66412233300952
148000 [-0.45619438  0.8599751 ] 0.47892512960097305 275.601082275046
149000 [-0.4419223   0.86724837] 0.4959366367619687 331.5988793213742
150000 [-0.470950

In [8]:
# run the NAMD code 'FEM/calc_q.edp' with sampled points 'coor_220.txt' to obtain FEM value.

# print the RMSE and MAE error
qFEM = np.loadtxt('FEM/q_fem.txt')
qNN  = sess.run(q,{x:cs})
print("RMSE: ",np.sqrt(np.mean((qFEM-qNN)**2)))
print("MAE: ",np.mean(np.abs(qFEM-qNN)))

RMSE:  0.01983819403014393
MAE:  0.01525394281304963
